In [1]:
import joblib
from statsmodels.stats.weightstats import ttest_ind
from statsmodels.stats.weightstats import DescrStatsW
import numpy as np
import pandas as pd
from ALSampler import summarize
from itertools import chain
from pathlib import Path 
pd.options.display.float_format='{:,.3f}'.format

In [43]:
m_scores=joblib.load(r'../data/n2c2/scores_sentence_sampling/ner_crf_scores_sentenceSampling_500bootstrap.joblib')
mfull_scores=joblib.load(r'../data/n2c2/scores_sentence_sampling/ner_crf_scores_sentenceSampling_full500bootstrap.joblib')
r_scores=joblib.load(r'../data/n2c2/scores_sentence_sampling/ner_random_scores_sentenceSampling_500bootstrap.joblib')
vmax_scores=joblib.load(r'../data/n2c2/scores_sentence_sampling/ner_VBmax_scores_sentenceSampling_500bootstrap2.joblib')
vmin_scores=joblib.load(r'../data/n2c2/scores_sentence_sampling/ner_VBmin_scores_sentenceSampling_500bootstrap.joblib')

In [3]:
name1='m'
name2='v'
measures=['precision','recall','f1'] 
cols=[b+"_"+a for b in measures for a in [name1, name2,'pv'] ]
results={col:[] for col in cols}

In [4]:
results

{'precision_m': [],
 'precision_v': [],
 'precision_pv': [],
 'recall_m': [],
 'recall_v': [],
 'recall_pv': [],
 'f1_m': [],
 'f1_v': [],
 'f1_pv': []}

In [44]:
def compute_stats(scores1, scores2, name1, name2, runs=3, rounds=10):
    measures=['precision','recall','f1'] 
    cols=[b+"_"+a for b in measures for a in [name1, name2,'pv'] ]
    results={col:[] for col in cols}
    for ri in range(rounds):
        single_round_scores={name1:{}, name2:{}}
        for m in measures:             
            # print('s1',len(scores1), len(scores1[0]), len(scores1[0][0][m]))
            # print('s2',len(scores2), len(scores2[0]), len(scores2[0][0][m]))
            single_round_scores[name1][m]=list(chain.from_iterable([scores1[u][ri][m] for u in range(runs)]))
            single_round_scores[name2][m]=list(chain.from_iterable([scores2[u][ri][m] for u in range(runs)]))
            results[m+'_'+name1].append(np.mean(single_round_scores[name1][m]))
            results[m+'_'+name2].append(np.mean(single_round_scores[name2][m]))
            ttres=ttest_ind(single_round_scores[name1][m], single_round_scores[name2][m], usevar='unequal')
            # ttres=DescrStatsW.ttest_mean(np.array(single_round_scores[name1][m])-np.array(single_round_scores[name2][m]))
            results[m+'_pv'].append(ttres[1])
    return pd.DataFrame(results)

In [6]:
list(range(3))

[0, 1, 2]

## model vs full uncertainty model

In [45]:
results=compute_stats(m_scores, mfull_scores, 'model', 'fullmodel')

In [46]:
results

,precision_model,precision_fullmodel,precision_pv,recall_model,recall_fullmodel,recall_pv,f1_model,f1_fullmodel,f1_pv
0,0.949,0.949,1.000,0.492,0.492,1.000,0.648,0.648,1.000
1,0.878,0.928,0.000,0.703,0.624,0.000,0.780,0.746,0.000
2,0.911,0.933,0.000,0.734,0.650,0.000,0.813,0.766,0.000
3,0.909,0.926,0.000,0.769,0.687,0.000,0.833,0.789,0.000
4,0.919,0.933,0.000,0.781,0.710,0.000,0.844,0.806,0.000
5,0.920,0.932,0.000,0.794,0.734,0.000,0.852,0.821,0.000
6,0.922,0.932,0.000,0.804,0.751,0.000,0.859,0.832,0.000
7,0.922,0.934,0.000,0.814,0.764,0.000,0.865,0.840,0.000
8,0.924,0.934,0.000,0.819,0.780,0.000,0.868,0.851,0.000
9,0.926,0.933,0.000,0.823,0.793,0.000,0.871,0.857,0.000


### model vs vmin

In [31]:
results=compute_stats(m_scores, vmin_scores, 'model', 'vmin')

In [32]:
results

,precision_model,precision_vmin,precision_pv,recall_model,recall_vmin,recall_pv,f1_model,f1_vmin,f1_pv
0,0.949,0.949,1.000,0.492,0.492,1.000,0.648,0.648,1.000
1,0.878,0.920,0.000,0.703,0.613,0.000,0.780,0.736,0.000
2,0.911,0.930,0.000,0.734,0.619,0.000,0.813,0.743,0.000
3,0.909,0.937,0.000,0.769,0.652,0.000,0.833,0.769,0.000
4,0.919,0.938,0.000,0.781,0.687,0.000,0.844,0.793,0.000
5,0.920,0.940,0.000,0.794,0.697,0.000,0.852,0.801,0.000
6,0.922,0.938,0.000,0.804,0.714,0.000,0.859,0.811,0.000
7,0.922,0.938,0.000,0.814,0.723,0.000,0.865,0.817,0.000
8,0.924,0.935,0.000,0.819,0.737,0.000,0.868,0.824,0.000
9,0.926,0.935,0.000,0.823,0.749,0.000,0.871,0.832,0.000


### model vs vmax

In [39]:
results=compute_stats(m_scores, vmax_scores, 'model', 'vmax')
results

,precision_model,precision_vmax,precision_pv,recall_model,recall_vmax,recall_pv,f1_model,f1_vmax,f1_pv
0,0.949,0.949,1.000,0.492,0.492,1.000,0.648,0.648,1.000
1,0.878,0.922,0.000,0.703,0.609,0.000,0.780,0.733,0.000
2,0.911,0.926,0.000,0.734,0.625,0.000,0.813,0.746,0.000
3,0.909,0.934,0.000,0.769,0.653,0.000,0.833,0.769,0.000
4,0.919,0.936,0.000,0.781,0.688,0.000,0.844,0.793,0.000
5,0.920,0.938,0.000,0.794,0.699,0.000,0.852,0.801,0.000
6,0.922,0.936,0.000,0.804,0.718,0.000,0.859,0.812,0.000
7,0.922,0.937,0.000,0.814,0.725,0.000,0.865,0.817,0.000
8,0.924,0.932,0.000,0.819,0.740,0.000,0.868,0.825,0.000
9,0.926,0.934,0.000,0.823,0.753,0.000,0.871,0.833,0.000


### model vs random

In [34]:
results=compute_stats(m_scores, r_scores, 'model', 'rand')
results

,precision_model,precision_rand,precision_pv,recall_model,recall_rand,recall_pv,f1_model,f1_rand,f1_pv
0,0.949,0.949,1.000,0.492,0.492,1.000,0.648,0.648,1.000
1,0.878,0.943,0.000,0.703,0.579,0.000,0.780,0.717,0.000
2,0.911,0.942,0.000,0.734,0.609,0.000,0.813,0.740,0.000
3,0.909,0.941,0.000,0.769,0.642,0.000,0.833,0.763,0.000
4,0.919,0.943,0.000,0.781,0.661,0.000,0.844,0.777,0.000
5,0.920,0.944,0.000,0.794,0.678,0.000,0.852,0.789,0.000
6,0.922,0.944,0.000,0.804,0.692,0.000,0.859,0.799,0.000
7,0.922,0.943,0.000,0.814,0.705,0.000,0.865,0.807,0.000
8,0.924,0.943,0.000,0.819,0.713,0.000,0.868,0.812,0.000
9,0.926,0.943,0.000,0.823,0.723,0.000,0.871,0.819,0.000


### vmin vs random

In [41]:
results=compute_stats(vmin_scores, r_scores,  'vmin', 'rand')
results

,precision_vmin,precision_rand,precision_pv,recall_vmin,recall_rand,recall_pv,f1_vmin,f1_rand,f1_pv
0,0.949,0.949,1.000,0.492,0.492,1.000,0.648,0.648,1.000
1,0.920,0.943,0.000,0.613,0.579,0.000,0.736,0.717,0.000
2,0.930,0.942,0.000,0.619,0.609,0.000,0.743,0.740,0.000
3,0.937,0.941,0.000,0.652,0.642,0.000,0.769,0.763,0.000
4,0.938,0.943,0.000,0.687,0.661,0.000,0.793,0.777,0.000
5,0.940,0.944,0.000,0.697,0.678,0.000,0.801,0.789,0.000
6,0.938,0.944,0.000,0.714,0.692,0.000,0.811,0.799,0.000
7,0.938,0.943,0.000,0.723,0.705,0.000,0.817,0.807,0.000
8,0.935,0.943,0.000,0.737,0.713,0.000,0.824,0.812,0.000
9,0.935,0.943,0.000,0.749,0.723,0.000,0.832,0.819,0.000


### vmax vs random

In [40]:
results=compute_stats(vmax_scores,r_scores,  'vmax', 'rand')
results

,precision_vmax,precision_rand,precision_pv,recall_vmax,recall_rand,recall_pv,f1_vmax,f1_rand,f1_pv
0,0.949,0.949,1.000,0.492,0.492,1.000,0.648,0.648,1.000
1,0.922,0.943,0.000,0.609,0.579,0.000,0.733,0.717,0.000
2,0.926,0.942,0.000,0.625,0.609,0.000,0.746,0.740,0.000
3,0.934,0.941,0.000,0.653,0.642,0.000,0.769,0.763,0.000
4,0.936,0.943,0.000,0.688,0.661,0.000,0.793,0.777,0.000
5,0.938,0.944,0.000,0.699,0.678,0.000,0.801,0.789,0.000
6,0.936,0.944,0.000,0.718,0.692,0.000,0.812,0.799,0.000
7,0.937,0.943,0.000,0.725,0.705,0.000,0.817,0.807,0.000
8,0.932,0.943,0.000,0.740,0.713,0.000,0.825,0.812,0.000
9,0.934,0.943,0.000,0.753,0.723,0.000,0.833,0.819,0.000


### vmax vs vmin

In [47]:
results=compute_stats(vmin_scores, vmax_scores, 'vmin', 'vmax')
results

,precision_vmin,precision_vmax,precision_pv,recall_vmin,recall_vmax,recall_pv,f1_vmin,f1_vmax,f1_pv
0,0.949,0.949,1.000,0.492,0.492,1.000,0.648,0.648,1.000
1,0.920,0.922,0.000,0.613,0.609,0.000,0.736,0.733,0.000
2,0.930,0.926,0.000,0.619,0.625,0.000,0.743,0.746,0.000
3,0.937,0.934,0.000,0.652,0.653,0.000,0.769,0.769,0.064
4,0.938,0.936,0.000,0.687,0.688,0.000,0.793,0.793,0.005
5,0.940,0.938,0.000,0.697,0.699,0.000,0.801,0.801,0.688
6,0.938,0.936,0.000,0.714,0.718,0.000,0.811,0.812,0.000
7,0.938,0.937,0.000,0.723,0.725,0.000,0.817,0.817,0.000
8,0.935,0.932,0.000,0.737,0.740,0.000,0.824,0.825,0.000
9,0.935,0.934,0.000,0.749,0.753,0.000,0.832,0.833,0.000
